## data process

In [23]:
import pandas as pd
import os

base_path = "/ihome/ylee/yiz133/Code/Scirpy/scripy/data"
cns_path = f"{base_path}/0516_CNS_filtered_contig_annotations.csv"
spleen_path= f"{base_path}/0516_SPL_filtered_contig_annotations.csv"

# Strict filtering for the TCRs based on scirpy analysis
filtered_meta = pd.read_csv(f"{base_path}/strict_filtered_meta.csv")
barcodes_set = set(filtered_meta.iloc[:,0])
cns_0605_ids = {b.split("_")[0] for b in barcodes_set if "CNS" in b}
spleen_0605_ids = {b.split("_")[0] for b in barcodes_set if "SPL" in b}


# Define output directories
output_dir = "/ihome/ylee/yiz133/Code/deepTCR/unSupervised_cluster/processed_data_0516"
cns_dir = os.path.join(output_dir, "CNS")
spleen_dir = os.path.join(output_dir, "Spleen")

# Create folders if they don't exist
os.makedirs(cns_dir, exist_ok=True)
os.makedirs(spleen_dir, exist_ok=True)

# Function to filter for high-confidence, productive TCR Beta (TRB) chains
def strict_filter_tcr_beta(df, ids, sample):
    df["barcode"] = df["barcode"] + sample
    df = df[
        (df["high_confidence"] == True) &
        (df["productive"] == True) &
        (df["chain"] == "TRB")
    ][["cdr3", "v_gene","j_gene", "barcode"]].dropna()

    # Remove multiple V-Beta/J-Beta notations (keep only the first one if multiple exist)
    df["v_gene"] = df["v_gene"].apply(lambda x: x.split("+")[0] if "+" in x else x)
    df["j_gene"] = df["j_gene"].apply(lambda x: x.split("+")[0] if "+" in x else x)

    return df

# Apply filtering
cns_filtered = strict_filter_tcr_beta(pd.read_csv(cns_path), cns_0605_ids, '_0605_CNS')
spleen_filtered = strict_filter_tcr_beta(pd.read_csv(spleen_path), spleen_0605_ids, '_0605_SPL')

In [24]:
# Add count column (each row initially counts as 1)
cns_filtered["count"] = 1
spleen_filtered["count"] = 1

# Resolve V-Beta/J-Beta conflicts by selecting the most common one per CDR3
def resolve_vj_conflicts(group):
    v_gene = group["v_gene"].value_counts().idxmax() if len(group["v_gene"].unique()) > 1 else group["v_gene"].iloc[0]
    j_gene = group["j_gene"].value_counts().idxmax() if len(group["j_gene"].unique()) > 1 else group["j_gene"].iloc[0]
    count = group["count"].sum()  # Sum counts for duplicate CDR3s
    return pd.Series([count, v_gene, j_gene], index=["count", "v_gene", "j_gene"])

# Process CNS and Spleen separately
cns_final = cns_filtered.groupby("cdr3").apply(resolve_vj_conflicts).reset_index()
spleen_final = spleen_filtered.groupby("cdr3").apply(resolve_vj_conflicts).reset_index()

# Reorder columns to match format: beta, counts, v_beta, j_beta
cns_final = cns_final[["cdr3", "count", "v_gene", "j_gene"]]
spleen_final = spleen_final[["cdr3", "count", "v_gene", "j_gene"]]

# Save CNS and Spleen data separately
cns_output_file = os.path.join(cns_dir, "CNS.tsv")
spleen_output_file = os.path.join(spleen_dir, "Spleen.tsv")

# Ensure headers are included
cns_final.to_csv(cns_output_file, sep="\t", index=False, header=["beta", "counts", "v_beta", "j_beta"])
spleen_final.to_csv(spleen_output_file, sep="\t", index=False, header=["beta", "counts", "v_beta", "j_beta"])

print(f"Saved CNS sequences to: {cns_output_file}")
print(f"Saved Spleen sequences to: {spleen_output_file}")

/scratch/slurm-6011815/ipykernel_30046/1037160698.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cns_final = cns_filtered.groupby("cdr3").apply(resolve_vj_conflicts).reset_index()


Saved CNS sequences to: /ihome/ylee/yiz133/Code/deepTCR/unSupervised_cluster/processed_data_0516/CNS/CNS.tsv
Saved Spleen sequences to: /ihome/ylee/yiz133/Code/deepTCR/unSupervised_cluster/processed_data_0516/Spleen/Spleen.tsv


/scratch/slurm-6011815/ipykernel_30046/1037160698.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spleen_final = spleen_filtered.groupby("cdr3").apply(resolve_vj_conflicts).reset_index()


# 0516

In [25]:
import pandas as pd
import os

base_path = "/ihome/ylee/yiz133/Code/Scirpy/scripy/data"
cns_path = f"{base_path}/0516_CNS_filtered_contig_annotations.csv"
spleen_path= f"{base_path}/0516_SPL_filtered_contig_annotations.csv"

cns_raw = pd.read_csv(cns_path)
spl_raw = pd.read_csv(spleen_path)

In [26]:
# Define output directories
output_dir = "/ihome/ylee/yiz133/Code/deepTCR/unSupervised_cluster/processed_data_0516"
cns_dir = os.path.join(output_dir, "CNS")
spleen_dir = os.path.join(output_dir, "Spleen")

# Create folders if they don't exist
os.makedirs(cns_dir, exist_ok=True)
os.makedirs(spleen_dir, exist_ok=True)

# Function to filter for high-confidence, productive TCR Beta (TRB) chains
def strict_filter_tcr_beta(df, sample):
    df["barcode"] = df["barcode"] + sample
    df = df[
        (df["high_confidence"] == True) &
        (df["productive"] == True) &
        (df["chain"] == "TRB")
    ][["cdr3", "v_gene","j_gene", "barcode"]].dropna()

    # Remove multiple V-Beta/J-Beta notations (keep only the first one if multiple exist)
    df["v_gene"] = df["v_gene"].apply(lambda x: x.split("+")[0] if "+" in x else x)
    df["j_gene"] = df["j_gene"].apply(lambda x: x.split("+")[0] if "+" in x else x)

    return df

# Apply filtering
cns_filtered = strict_filter_tcr_beta(cns_raw, "_0516_CNS")
spleen_filtered = strict_filter_tcr_beta(spl_raw, "_0516_SPL")

In [27]:
# Add count column (each row initially counts as 1)
cns_filtered["count"] = 1
spleen_filtered["count"] = 1

# Resolve V-Beta/J-Beta conflicts by selecting the most common one per CDR3
def resolve_vj_conflicts(group):
    v_gene = group["v_gene"].value_counts().idxmax() if len(group["v_gene"].unique()) > 1 else group["v_gene"].iloc[0]
    j_gene = group["j_gene"].value_counts().idxmax() if len(group["j_gene"].unique()) > 1 else group["j_gene"].iloc[0]
    count = group["count"].sum()  # Sum counts for duplicate CDR3s
    return pd.Series([count, v_gene, j_gene], index=["count", "v_gene", "j_gene"])

# Process CNS and Spleen separately
cns_final = cns_filtered.groupby("cdr3").apply(resolve_vj_conflicts).reset_index()
spleen_final = spleen_filtered.groupby("cdr3").apply(resolve_vj_conflicts).reset_index()

/scratch/slurm-6011815/ipykernel_30046/2008921118.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cns_final = cns_filtered.groupby("cdr3").apply(resolve_vj_conflicts).reset_index()
/scratch/slurm-6011815/ipykernel_30046/2008921118.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spleen_final = spleen_filtered.groupby("cdr3").apply(resolve_vj_conflicts).reset_index()


In [28]:
# Reorder columns to match format: beta, counts, v_beta, j_beta
cns_final = cns_final[["cdr3", "count", "v_gene", "j_gene"]]
spleen_final = spleen_final[["cdr3", "count", "v_gene", "j_gene"]]

# Save CNS and Spleen data separately
cns_output_file = os.path.join(cns_dir, "CNS.tsv")
spleen_output_file = os.path.join(spleen_dir, "Spleen.tsv")

# Ensure headers are included
cns_final.to_csv(cns_output_file, sep="\t", index=False, header=["beta", "counts", "v_beta", "j_beta"])
spleen_final.to_csv(spleen_output_file, sep="\t", index=False, header=["beta", "counts", "v_beta", "j_beta"])

print(f"Saved CNS sequences to: {cns_output_file}")
print(f"Saved Spleen sequences to: {spleen_output_file}")

Saved CNS sequences to: /ihome/ylee/yiz133/Code/deepTCR/unSupervised_cluster/processed_data_0516/CNS/CNS.tsv
Saved Spleen sequences to: /ihome/ylee/yiz133/Code/deepTCR/unSupervised_cluster/processed_data_0516/Spleen/Spleen.tsv
